In [25]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
train = pd.read_csv("train.csv", index_col=[0], parse_dates=[0])
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

In [3]:
train

,Junction,Vehicles,ID
DateTime,,,
2015-11-01 00:00:00,1,15,20151101001
2015-11-01 01:00:00,1,13,20151101011
2015-11-01 02:00:00,1,10,20151101021
2015-11-01 03:00:00,1,7,20151101031
2015-11-01 04:00:00,1,9,20151101041
...,...,...,...
2017-06-30 19:00:00,4,11,20170630194
2017-06-30 20:00:00,4,30,20170630204
2017-06-30 21:00:00,4,16,20170630214


In [4]:
jun1 = train[:14592]
jun2 = train[14592:29184]
jun3 = train[29184:43776]
jun4 = train[43776:]

In [5]:
y1 = jun1['Vehicles']
y2 = jun2['Vehicles']
y3 = jun3['Vehicles']
y4 = jun4['Vehicles']

jun1 = jun1.drop(['Vehicles'], axis=1)
jun2 = jun2.drop(['Vehicles'], axis=1)
jun3 = jun3.drop(['Vehicles'], axis=1)
jun4 = jun4.drop(['Vehicles'], axis=1)

In [6]:
jun1_test = test[:2952]
jun2_test = test[2952:5904]
jun3_test = test[5904:8856]
jun4_test = test[8856:]

jun1_test = jun1_test.set_index('DateTime')
jun2_test = jun2_test.set_index('DateTime')
jun3_test = jun3_test.set_index('DateTime')
jun4_test = jun4_test.set_index('DateTime')

In [7]:
ID1 = jun1_test['ID']
ID2 = jun2_test['ID']
ID3 = jun3_test['ID']
ID4 = jun4_test['ID']

In [8]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index.
    """
    df = df.copy()
    df['date'] = df.index
    df['date'] = pd.to_datetime(df['date'])
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear
    
    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X

X1 = create_features(jun1, label=None)
Y1 = create_features(jun1_test, label=None)

X2 = create_features(jun2, label=None)
Y2 = create_features(jun2_test, label=None)

X3 = create_features(jun3, label=None)
Y3 = create_features(jun3_test, label=None)

X4 = create_features(jun4, label=None)
Y4 = create_features(jun4_test, label=None)

In [9]:
data1= X1[['hour', 'dayofweek', 'quarter', 'month', 'year','dayofyear','dayofmonth','weekofyear']]
data2= X2[['hour', 'dayofweek', 'quarter', 'month', 'year','dayofyear','dayofmonth','weekofyear']]
data3= X3[['hour', 'dayofweek', 'quarter', 'month', 'year','dayofyear','dayofmonth','weekofyear']]
data4= X4[['hour', 'dayofweek', 'quarter', 'month', 'year','dayofyear','dayofmonth','weekofyear']]

In [10]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'error', math.sqrt(mean_squared_error(labels, preds))

In [11]:
import xgboost as xgb
from xgboost import XGBClassifier
import math
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [12]:
# # split data to training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.02, random_state=1)#28776
# # convert data to Dmatrix format for xgboost
# X_train
d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_test, label=y_test)
d_test1 = xgb.DMatrix(Y1)
params = {  
    "n_estimators": 300 ,  
    "max_depth": 4 ,                  
    "learning_rate": 0.01 ,    
    "colsample_bytree": 1 ,     
    "subsample": 1 ,            
    "gamma": 0.1 ,              
    'reg_alpha': 10 ,           
    "min_child_weight": 4 ,     
    }

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

# Train xgboost regressor
reg1 = xgb.train(params, d_train, 5000, watchlist,  feval = evalerror, maximize=False, verbose_eval=50)
# Predict from xgboost regressor
Vehicles_valid1 = reg1.predict(d_valid)

[15:17:33] WARNING: /workspace/src/learner.cc:480: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:49.68873	valid-rmse:48.45998	train-error:49.68872	valid-error:48.45998
[50]	train-rmse:31.13142	valid-rmse:30.57539	train-error:31.13143	valid-error:30.57539
[100]	train-rmse:20.11042	valid-rmse:19.94264	train-error:20.11042	valid-error:19.94264
[150]	train-rmse:13.68447	valid-rmse:13.75173	train-error:13.68447	valid-error:13.75173
[200]	train-rmse:10.04360	valid-rmse:10.26558	train-error:10.04360	valid-error:10.26558
[250]	train-rmse:8.02126	valid-rmse:8.38670	train-error:8.02126	valid-error:8.38670
[300]	train-rmse:6.89996	valid-rmse:7.39490	train-error:6.89996	valid-error:7.39490
[350]	train-rmse:6.26886	valid-rmse:6.84986	train-error

[4650]	train-rmse:3.67705	valid-rmse:4.21601	train-error:3.67705	valid-error:4.21601
[4700]	train-rmse:3.67189	valid-rmse:4.21551	train-error:3.67189	valid-error:4.21551
[4750]	train-rmse:3.66672	valid-rmse:4.21324	train-error:3.66672	valid-error:4.21324
[4800]	train-rmse:3.66191	valid-rmse:4.21195	train-error:3.66191	valid-error:4.21195
[4850]	train-rmse:3.65791	valid-rmse:4.20995	train-error:3.65791	valid-error:4.20995
[4900]	train-rmse:3.65213	valid-rmse:4.20801	train-error:3.65213	valid-error:4.20801
[4950]	train-rmse:3.64714	valid-rmse:4.20529	train-error:3.64714	valid-error:4.20529
[4999]	train-rmse:3.64090	valid-rmse:4.20439	train-error:3.64090	valid-error:4.20439


In [13]:
# split data to training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X2, y2, test_size=0.02, random_state=1)#28776
# convert data to Dmatrix format for xgboost
d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_test, label=y_test)
d_test2 = xgb.DMatrix(Y2)
params = {  
    "n_estimators": 300 ,  
    "max_depth": 4 ,                  
    "learning_rate": 0.005 ,    
    "colsample_bytree": 1 ,     
    "subsample": 1 ,            
    "gamma": 0.1 ,              
    'reg_alpha': 10 ,           
    "min_child_weight": 4 ,     
    }

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

# Train xgboost regressor
reg2 = xgb.train(params, d_train, 5000, watchlist,  feval = evalerror, maximize=False, verbose_eval=50)
# Predict from xgboost regressor
Vehicles_valid2 = reg2.predict(d_valid)

[15:18:16] WARNING: /workspace/src/learner.cc:480: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:15.56136	valid-rmse:14.68441	train-error:15.56136	valid-error:14.68441
[50]	train-rmse:12.32374	valid-rmse:11.57256	train-error:12.32372	valid-error:11.57256
[100]	train-rmse:9.84271	valid-rmse:9.19742	train-error:9.84272	valid-error:9.19742
[150]	train-rmse:7.95271	valid-rmse:7.38346	train-error:7.95272	valid-error:7.38346
[200]	train-rmse:6.51628	valid-rmse:6.02185	train-error:6.51628	valid-error:6.02185
[250]	train-rmse:5.42964	valid-rmse:4.98650	train-error:5.42964	valid-error:4.98650
[300]	train-rmse:4.62336	valid-rmse:4.21971	train-error:4.62336	valid-error:4.21971
[350]	train-rmse:4.03707	valid-rmse:3.67119	train-error:4.03707	val

[4650]	train-rmse:2.09337	valid-rmse:2.13459	train-error:2.09337	valid-error:2.13459
[4700]	train-rmse:2.09178	valid-rmse:2.13396	train-error:2.09178	valid-error:2.13396
[4750]	train-rmse:2.09013	valid-rmse:2.13304	train-error:2.09013	valid-error:2.13304
[4800]	train-rmse:2.08867	valid-rmse:2.13250	train-error:2.08867	valid-error:2.13250
[4850]	train-rmse:2.08723	valid-rmse:2.13188	train-error:2.08723	valid-error:2.13188
[4900]	train-rmse:2.08573	valid-rmse:2.13018	train-error:2.08573	valid-error:2.13018
[4950]	train-rmse:2.08399	valid-rmse:2.12911	train-error:2.08399	valid-error:2.12911
[4999]	train-rmse:2.08274	valid-rmse:2.12833	train-error:2.08274	valid-error:2.12833


In [14]:
# split data to training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X3, y3, test_size=0.02, random_state=1)#28776
# convert data to Dmatrix format for xgboost
d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_test, label=y_test)
d_test3 = xgb.DMatrix(Y3)
params = {  
    "n_estimators": 300 ,  
    "max_depth": 4 ,                  
    "learning_rate": 0.005 ,    
    "colsample_bytree": 1 ,     
    "subsample": 1 ,            
    "gamma": 0.1 ,              
    'reg_alpha': 10 ,           
    "min_child_weight": 4 ,     
    }

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

# Train xgboost regressor
reg3 = xgb.train(params, d_train, 5000, watchlist,  feval = evalerror, maximize=False, verbose_eval=50)
# Predict from xgboost regressor
Vehicles_valid3 = reg3.predict(d_valid)

[15:19:01] WARNING: /workspace/src/learner.cc:480: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:16.75935	valid-rmse:16.75497	train-error:16.75934	valid-error:16.75498
[50]	train-rmse:14.07478	valid-rmse:14.07779	train-error:14.07479	valid-error:14.07779
[100]	train-rmse:12.15161	valid-rmse:12.16169	train-error:12.15161	valid-error:12.16169
[150]	train-rmse:10.80709	valid-rmse:10.80778	train-error:10.80709	valid-error:10.80778
[200]	train-rmse:9.89037	valid-rmse:9.86804	train-error:9.89037	valid-error:9.86804
[250]	train-rmse:9.25282	valid-rmse:9.18997	train-error:9.25282	valid-error:9.18997
[300]	train-rmse:8.81807	valid-rmse:8.72117	train-error:8.81808	valid-error:8.72117
[350]	train-rmse:8.50471	valid-rmse:8.38042	train-error:8.5

[4650]	train-rmse:5.75429	valid-rmse:5.84190	train-error:5.75429	valid-error:5.84190
[4700]	train-rmse:5.74737	valid-rmse:5.83550	train-error:5.74737	valid-error:5.83550
[4750]	train-rmse:5.73943	valid-rmse:5.82605	train-error:5.73943	valid-error:5.82605
[4800]	train-rmse:5.73189	valid-rmse:5.81041	train-error:5.73189	valid-error:5.81041
[4850]	train-rmse:5.72331	valid-rmse:5.79882	train-error:5.72331	valid-error:5.79882
[4900]	train-rmse:5.71492	valid-rmse:5.78760	train-error:5.71492	valid-error:5.78760
[4950]	train-rmse:5.70704	valid-rmse:5.78333	train-error:5.70704	valid-error:5.78333
[4999]	train-rmse:5.69805	valid-rmse:5.77799	train-error:5.69805	valid-error:5.77799


In [24]:
# split data to training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X4, y4, test_size=0.02, random_state=1)#28776
# convert data to Dmatrix format for xgboost
d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_test, label=y_test)
d_test4 = xgb.DMatrix(Y4)
params = {  
    "n_estimators": 300 ,  
    "max_depth": 4 ,                  
    "learning_rate": 0.005 ,    
    "colsample_bytree": 1 ,     
    "subsample": 1 ,            
    "gamma": 0.1 ,              
    'reg_alpha': 10 ,           
    "min_child_weight": 4 ,     
    }

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

# Train xgboost regressor
reg4 = xgb.train(params, d_train, 3000, watchlist,  feval = evalerror, maximize=False, verbose_eval=50)
# Predict from xgboost regressor
Vehicles_valid4 = reg4.predict(d_valid)

[15:20:57] WARNING: /workspace/src/learner.cc:480: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:7.58444	valid-rmse:7.42284	train-error:7.58444	valid-error:7.42284
[50]	train-rmse:6.13220	valid-rmse:5.98934	train-error:6.13220	valid-error:5.98934
[100]	train-rmse:5.04431	valid-rmse:4.89364	train-error:5.04430	valid-error:4.89364
[150]	train-rmse:4.24430	valid-rmse:4.08406	train-error:4.24430	valid-error:4.08406
[200]	train-rmse:3.66949	valid-rmse:3.49403	train-error:3.66949	valid-error:3.49403
[250]	train-rmse:3.26383	valid-rmse:3.08243	train-error:3.26383	valid-error:3.08243
[300]	train-rmse:2.98405	valid-rmse:2.79412	train-error:2.98405	valid-error:2.79412
[350]	train-rmse:2.79257	valid-rmse:2.59712	train-error:2.79257	valid-error

In [16]:
jun1_prediction = reg1.predict(d_test1)
jun2_prediction = reg2.predict(d_test2)
jun3_prediction = reg3.predict(d_test3)
jun4_prediction = reg4.predict(d_test4)

In [17]:
pred1 = pd.DataFrame({
    "ID":ID1,
    "Vehicles": jun1_prediction
})

In [18]:
pred2 = pd.DataFrame({
    "ID":ID2,
    "Vehicles": jun2_prediction
})

In [19]:
pred3 = pd.DataFrame({
    "ID":ID3,
    "Vehicles": jun3_prediction
})

In [20]:
pred4 = pd.DataFrame({
    "ID":ID4,
    "Vehicles": jun4_prediction
})

In [21]:
pred = pd.concat([pred1, pred2, pred3, pred4])

In [22]:
# pred.to_csv("submission2.csv", index=False)